1. http://www.manythings.org/anki 
 - 두 언어 간 병렬 코퍼스 다운 가능
 - en-fra : 10만개 이상 데이터 존재 -> 3만개만 사용
 - en-kor : 3천개 존재 -> 시도해봤는데 결과가 그닥...

2. 과제
 - 오늘 실습한 코드를 참고해서 peeky 모델 구현
 - 사용하고 싶은 데이터는 어떤 것을 사용해도 무관
 - 17기_OOO_seq2seq.ipynb 형태로 제출해주세요~

In [1]:
import pandas as pd
import numpy as np
import re
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer

In [2]:
from google.colab import files 
files.upload() 

Saving kor.txt to kor.txt


{'kor.txt': b'Go.\t\xea\xb0\x80.\tCC-BY 2.0 (France) Attribution: tatoeba.org #2877272 (CM) & #8363271 (Eunhee)\nHi.\t\xec\x95\x88\xeb\x85\x95.\tCC-BY 2.0 (France) Attribution: tatoeba.org #538123 (CM) & #8355888 (Eunhee)\nRun!\t\xeb\x9b\xb0\xec\x96\xb4!\tCC-BY 2.0 (France) Attribution: tatoeba.org #906328 (papabear) & #8355891 (Eunhee)\nRun.\t\xeb\x9b\xb0\xec\x96\xb4.\tCC-BY 2.0 (France) Attribution: tatoeba.org #4008918 (JSakuragi) & #8363273 (Eunhee)\nWho?\t\xeb\x88\x84\xea\xb5\xac?\tCC-BY 2.0 (France) Attribution: tatoeba.org #2083030 (CK) & #6820074 (yesjustryan)\nWow!\t\xec\x9a\xb0\xec\x99\x80!\tCC-BY 2.0 (France) Attribution: tatoeba.org #52027 (Zifre) & #8355878 (Eunhee)\nFire!\t\xec\x8f\xb4!\tCC-BY 2.0 (France) Attribution: tatoeba.org #1829639 (Spamster) & #8355899 (Eunhee)\nHelp!\t\xeb\x8f\x84\xec\x99\x80\xec\xa4\x98!\tCC-BY 2.0 (France) Attribution: tatoeba.org #435084 (lukaszpp) & #8355885 (Eunhee)\nJump!\t\xec\xa0\x90\xed\x94\x84!\tCC-BY 2.0 (France) Attribution: tatoeba.

In [4]:
lines = pd.read_csv('kor.txt', names=['src', 'tar', 'lic'], sep='\t',encoding='utf-8')
del lines['lic']
len(lines)

3648

In [5]:
lines.head()

,src,tar
0,Go.,가.
1,Hi.,안녕.
2,Run!,뛰어!
3,Run.,뛰어.
4,Who?,누구?


## 1. Preprocessing

In [6]:
lines['src'] = lines['src'].apply(lambda x : re.sub(r"([?.!,¿])", r" \1", x))
lines['src'] = lines['src'].apply(lambda x : x.lower())
lines['tar'] = lines['tar'].apply(lambda x : re.sub(r"([?.!,¿])", r" \1", x))

lines.head()

,src,tar
0,go .,가 .
1,hi .,안녕 .
2,run !,뛰어 !
3,run .,뛰어 .
4,who ?,누구 ?


In [7]:
src_input = []
for sample in lines['src'].to_numpy():
  src_input.append(sample.split())

tar_input = []
for sample in lines['tar'].to_numpy():
  tar_input.append(("<sos> " + sample).split())

tar_target = []
for sample in lines['tar'].to_numpy():
  tar_target.append((sample + " <eos>").split())

In [8]:
print(src_input[:5])
print(tar_input[:5])
print(tar_target[:5])

[['go', '.'], ['hi', '.'], ['run', '!'], ['run', '.'], ['who', '?']]
[['<sos>', '가', '.'], ['<sos>', '안녕', '.'], ['<sos>', '뛰어', '!'], ['<sos>', '뛰어', '.'], ['<sos>', '누구', '?']]
[['가', '.', '<eos>'], ['안녕', '.', '<eos>'], ['뛰어', '!', '<eos>'], ['뛰어', '.', '<eos>'], ['누구', '?', '<eos>']]


In [9]:
tokenizer_src = Tokenizer()
tokenizer_src.fit_on_texts(src_input)
encoder_input = tokenizer_src.texts_to_sequences(src_input)

tokenizer_tar = Tokenizer()
tokenizer_tar.fit_on_texts(tar_input)
tokenizer_tar.fit_on_texts(tar_target)
decoder_input = tokenizer_tar.texts_to_sequences(tar_input)
decoder_target = tokenizer_tar.texts_to_sequences(tar_target)

In [10]:
src_vocab_size = len(tokenizer_src.word_index) + 1
tar_vocab_size = len(tokenizer_tar.word_index) + 1
print("src voca size : {:d}, tar voca size : {:d}".format(src_vocab_size, tar_vocab_size))

src voca size : 2555, tar voca size : 5553


In [11]:
encoder_input = pad_sequences(encoder_input, padding="post")
decoder_input = pad_sequences(decoder_input, padding="post")
decoder_target = pad_sequences(decoder_target, padding="post")

In [12]:
indices = np.arange(encoder_input.shape[0])
np.random.shuffle(indices)

encoder_input = encoder_input[indices]
decoder_input = decoder_input[indices]
decoder_target = decoder_target[indices]

In [13]:
print(encoder_input.shape)
print(decoder_input.shape)
print(decoder_target.shape)

(3648, 110)
(3648, 96)
(3648, 96)


In [14]:
print(encoder_input[10])
print(decoder_input[10])
print(decoder_target[10])

[   3  178   43 1008    1    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0]
[   2    9  200 2240 2241    6    1    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0

In [15]:
n_of_val = 300
encoder_input_train = encoder_input[:-n_of_val]
decoder_input_train = decoder_input[:-n_of_val]
decoder_target_train = decoder_target[:-n_of_val]

encoder_input_test = encoder_input[-n_of_val:]
decoder_input_test = decoder_input[-n_of_val:]
decoder_target_test = decoder_target[-n_of_val:]

## 2. Modeling

In [16]:
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Masking
from tensorflow.keras.models import Model

latent_dim = 50

In [17]:
# encoder
encoder_inputs = Input(shape=(None,))
enc_emb =  Embedding(src_vocab_size, latent_dim)(encoder_inputs) 
encoder_lstm = LSTM(latent_dim, return_state=True) 
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb) 
encoder_states = [state_h, state_c] 

In [18]:
# decoder
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(tar_vocab_size, latent_dim) 
dec_emb = dec_emb_layer(decoder_inputs) 

decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True) 

decoder_outputs, _, _ = decoder_lstm(dec_emb, initial_state = encoder_states)

decoder_dense = Dense(tar_vocab_size, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [19]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy', metrics=['acc'])
model.fit(x = [encoder_input_train, decoder_input_train], y = decoder_target_train, \
          validation_data = ([encoder_input_test, decoder_input_test], decoder_target_test),
          batch_size = 32, epochs = 50)

Epoch 1/50
105/105 [==============================] - 41s 79ms/step - loss: 5.1832 - acc: 0.8889 - val_loss: 0.5425 - val_acc: 0.9368
Epoch 2/50
105/105 [==============================] - 6s 62ms/step - loss: 0.4817 - acc: 0.9352 - val_loss: 0.3889 - val_acc: 0.9471
Epoch 3/50
105/105 [==============================] - 6s 61ms/step - loss: 0.3761 - acc: 0.9461 - val_loss: 0.3743 - val_acc: 0.9564
Epoch 4/50
105/105 [==============================] - 6s 59ms/step - loss: 0.3596 - acc: 0.9540 - val_loss: 0.3670 - val_acc: 0.9567
Epoch 5/50
105/105 [==============================] - 6s 59ms/step - loss: 0.3490 - acc: 0.9547 - val_loss: 0.3628 - val_acc: 0.9567
Epoch 6/50
105/105 [==============================] - 6s 59ms/step - loss: 0.3377 - acc: 0.9549 - val_loss: 0.3625 - val_acc: 0.9564
Epoch 7/50
105/105 [==============================] - 6s 60ms/step - loss: 0.3374 - acc: 0.9544 - val_loss: 0.3607 - val_acc: 0.9566
Epoch 8/50
105/105 [==============================] - 6s 59ms/step -

## 3. Model test

In [20]:
# encoder for test
encoder_model = Model(encoder_inputs, encoder_states)

# decoder for test
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

dec_emb2= dec_emb_layer(decoder_inputs)

decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]

decoder_outputs2 = decoder_dense(decoder_outputs2)

decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)

In [21]:
src_to_index = tokenizer_src.word_index
index_to_src = tokenizer_src.index_word 

tar_to_index = tokenizer_tar.word_index
index_to_tar = tokenizer_tar.index_word 

In [22]:
def decode_sequence(input_seq):
    states_value = encoder_model.predict(input_seq)

    tar_seq = np.zeros((1,1))
    tar_seq[0, 0] = tar_to_index['<sos>']

    stop_condition = False
    decoded_sentence = ''

    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([tar_seq] + states_value)

        sampled_token_index = np.argmax(output_tokens[0, 0, :])
        sampled_char = index_to_tar[sampled_token_index]

        decoded_sentence += ' '+sampled_char

        if (sampled_char == '<eos>' or
           len(decoded_sentence) > 50):
            stop_condition = True

        tar_seq = np.zeros((1,1))
        tar_seq[0, 0] = sampled_token_index

        states_value = [h, c]

    return decoded_sentence

In [23]:
# 원문의 정수 시퀀스를 텍스트 시퀀스로 변환
def seq2src(input_seq):
    temp=''
    for i in input_seq:
        if(i!=0):
            temp = temp + index_to_src[i]+' '
    return temp

# 번역문의 정수 시퀀스를 텍스트 시퀀스로 변환
def seq2tar(input_seq):
    temp=''
    for i in input_seq:
        if((i!=0 and i!=tar_to_index['<sos>']) and i!=tar_to_index['<eos>']):
            temp = temp + index_to_tar[i] + ' '
    return temp

In [24]:
for seq_index in [3,50,100,300,1001]:
  input_seq = encoder_input[seq_index: seq_index + 1]
  decoded_sentence = decode_sequence(input_seq)

  print("원문 : ",seq2src(encoder_input[seq_index]))
  print("번역문 :",seq2tar(decoder_input[seq_index]))
  print("예측문 :",decoded_sentence[:-5])
  print("\n")

원문 :  there was one witness . 
번역문 : 목격자가 한 명 있었다 . 
예측문 :  넌 사람은 공부해야 수 있어 . 


원문 :  my boss thinks he's god . 
번역문 : 내 상사는 자기가 신인 줄 알아 . 
예측문 :  난 톰이 프랑스어를 할 수 없었어 . 


원문 :  i have a dictionary . 
번역문 : 나에게 사전이 있다 . 
예측문 :  그 사람은 사람들은 아직도 할 수 있어 . 


원문 :  our wedding is tomorrow . 
번역문 : 우리의 결혼식은 내일이다 . 
예측문 :  내가 사람들은 프랑스어를 할 수 없어 . 


원문 :  tom works for a tech company . 
번역문 : 톰은 기술 회사에 일하고 있어 . 
예측문 :  톰은 자기 프랑스어를 할 수 없었어 . 


